In [1]:
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import csv
import os
import random
import math
from matplotlib.pyplot import scatter
%matplotlib notebook

In [2]:
def tanh(x, derivative=False):
    if (derivative == True):
        return (1 - (np.tanh(x) ** 2))
    return np.tanh(x)

def relu(x, derivative=False):
    if (derivative == True):
        return 1. * (x > 0)
    return x * (x > 0)

def sigmoid(x, derivative=False):
    sigm = 1. / (1. + np.exp(-x))
    if derivative:
        return sigm * (1. - sigm)
    return sigm
    
def activation_function(x, derivative=False):
    return relu(x, derivative)

In [3]:
class NeuralNetwork:
    
    def __init__(self, neuron_list, learning_rate, shrink_rate):
        self.layer_count = len(neuron_list)
        self.neuron_list = neuron_list
        self.weights = [abs(np.random.randn(y, x)) / shrink_rate for x, y in zip(neuron_list[:-1], neuron_list[1:])]
        self.activations = [np.zeros((x,1)) for x in neuron_list]
        self.deltas = [np.zeros((x,1)) for x in neuron_list]
        self.learning_rate = learning_rate
        
    def forward_propogation(self, x):
        self.activations[0] = x      
        for i in range(self.layer_count-1):
            self.activations[i+1] = activation_function(np.dot(self.weights[i], self.activations[i]))
        return self.activations[-1]
    
    def compute_deltas(self,output_labels):
        # Compute last layers' activations
        self.deltas[-1] = 2*activation_function(np.dot(self.weights[-1], self.activations[-2]), True)*(output_labels-activation_function(self.activations[-1]))            
        
        # Compute all deltas in all layers
        for l in range(self.layer_count-2, 0, -1):
            np.dot(np.dot(np.transpose(self.weights[l]),self.deltas[l+1]), np.transpose(activation_function(np.dot(self.weights[l-1], self.activations[l-1]), True)))
            
    def back_propogation(self, output_labels):
        # Compute deltas
        self.compute_deltas(output_labels)
        
        # Update weights 
        for l in range(0,self.layer_count-1):
            self.weights[l] += self.learning_rate * np.dot(self.deltas[l+1], np.transpose(self.activations[l]))

                 
    def train(self, x, y, epoch):
        error = []
        for e in range(epoch):
            instance_error = 0
            print("Epoch: " + str(e))
            for i in range(len(x)):
                estimation = self.forward_propogation(x[i])
                print("Est: " + str(np.transpose(estimation)))
                print("Out: " + str(np.transpose(y[i])))
                print("")
                instance_error += 100 * np.sum(((estimation - y[i])**2) / (y[i]**2)) / len(estimation)
                self.back_propogation(y[i])
            print("Error: " + str(instance_error/ len(x)))
            error.append(instance_error / len(x))
        return error   
    
    def test(self, x, y):
        error = []
        instance_error = 0
        for i in range(len(x)):
            estimation = self.forward_propogation(x[i])
            print("Est: " + str(np.transpose(estimation)))
            print("Out: " + str(np.transpose(y[i])))
            print("")
            instance_error += 100 * np.sum(((estimation - y[i])*2) / (y[i]*2)) / len(estimation)
        print("Test Error: " + str(instance_error/ len(x)))
        error.append(instance_error / len(x))
        return error

In [4]:
'''
def shuffle(self,x,y):
    total = np.hstack((x,y))
    np.random.shuffle(total)
    b = total[:, 0:-np.shape(x)[1]]
    a = total[:, -np.shape(x)[1]:]
    return a, b
'''

'\ndef shuffle(self,x,y):\n    total = np.hstack((x,y))\n    np.random.shuffle(total)\n    b = total[:, 0:-np.shape(x)[1]]\n    a = total[:, -np.shape(x)[1]:]\n    return a, b\n'

In [ ]:
# Sample data for test
xx = np.array([8, 20])
yy = np.array([3, 15])
means = [xx.mean(), yy.mean()]  
stds = [xx.std() / 3, yy.std() / 3]
corr = 0.5         # correlation
covs = [[stds[0]**2          , stds[0]*stds[1]*corr], 
        [stds[0]*stds[1]*corr,           stds[1]**2]] 

m = np.random.multivariate_normal(means, covs, 200).T

xx = np.array([10, 50])
yy = np.array([4, 30])
means = [xx.mean(), yy.mean()]  
stds = [xx.std() / 3, yy.std() / 3]
corr = 0.7       # correlation
covs = [[stds[0]**2          , stds[0]*stds[1]*corr], 
        [stds[0]*stds[1]*corr,           stds[1]**2]] 

m2 = np.random.multivariate_normal(means, covs, 200).T

inp = []
for i in range(200):
    a1 =  [m2[0][i],  m[1][i]]
    a2 =  [m[0][i],  m2[1][i]]
    a3 = a1 + a2
    a3 /= np.linalg.norm(a3)
    inp.append(np.transpose([a3]))

In [ ]:
# Example training
np.set_printoptions(suppress=True)
neuron_list = [4,2,4]
nn = NeuralNetwork(neuron_list, 0.01, 10)
err = nn.train(inp,inp, 20)
plt.plot(err)
plt.show()